In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [4]:
# SETUP
columns = ['Full Name', 'Name', 'Age', 'Overall', 'Potential', 'Team', 'Nation', 'Contract Start', 'Contract End', 'Id', 'Height', 'Weight', 'Foot', 'Best Overall',
          'Best Position', 'Joined', 'Loan Date End', 'Value', 'Wage', 'Release Clause', 'Total Attacking', 'Crossing',
          'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Total Skill', 'Dribbling', 'Curve', 'FK Accuracy',
          'Long Passing', 'Ball Control', 'Total Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
          'Total Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Total Mentality', 'Aggression',
          'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Total Defending', 'Marking',
          'Standing Tackle', 'Sliding Tackle', 'Total Goalkeeping', 'GK Diving', 'GK Handling', 'GK Kicking',
          'GK Positioning', 'GK Reflexes', 'Total Stats', 'Base Stats', 'Weak Foot', 'Skill Moves', 'Attacking Work Rate',
          'Defensive Work Rate', 'International Reputation', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Positions', 'URL']
players = pd.DataFrame(columns=columns)

In [5]:
players.head()

,Full Name,Name,Age,Overall,Potential,Team,Nation,Contract Start,Contract End,Id,Height,Weight,Foot,Best Overall,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,Positions,URL


In [6]:
raw_html = simple_get('https://sofifa.com/players?&showCol%5B%5D=pi&showCol%5B%5D=ae&showCol%5B%5D=hi&showCol%5B%5D=wi&showCol%5B%5D=pf&showCol%5B%5D=oa&showCol%5B%5D=pt&showCol%5B%5D=bo&showCol%5B%5D=bp&showCol%5B%5D=gu&showCol%5B%5D=jt&showCol%5B%5D=le&showCol%5B%5D=vl&showCol%5B%5D=wg&showCol%5B%5D=rc&showCol%5B%5D=ta&showCol%5B%5D=cr&showCol%5B%5D=fi&showCol%5B%5D=he&showCol%5B%5D=sh&showCol%5B%5D=vo&showCol%5B%5D=ts&showCol%5B%5D=dr&showCol%5B%5D=cu&showCol%5B%5D=fr&showCol%5B%5D=lo&showCol%5B%5D=bl&showCol%5B%5D=to&showCol%5B%5D=ac&showCol%5B%5D=sp&showCol%5B%5D=ag&showCol%5B%5D=re&showCol%5B%5D=ba&showCol%5B%5D=tp&showCol%5B%5D=so&showCol%5B%5D=ju&showCol%5B%5D=st&showCol%5B%5D=sr&showCol%5B%5D=ln&showCol%5B%5D=te&showCol%5B%5D=ar&showCol%5B%5D=in&showCol%5B%5D=po&showCol%5B%5D=vi&showCol%5B%5D=pe&showCol%5B%5D=cm&showCol%5B%5D=td&showCol%5B%5D=ma&showCol%5B%5D=sa&showCol%5B%5D=sl&showCol%5B%5D=tg&showCol%5B%5D=gd&showCol%5B%5D=gh&showCol%5B%5D=gk&showCol%5B%5D=gp&showCol%5B%5D=gr&showCol%5B%5D=tt&showCol%5B%5D=bs&showCol%5B%5D=wk&showCol%5B%5D=sk&showCol%5B%5D=aw&showCol%5B%5D=dw&showCol%5B%5D=ir&showCol%5B%5D=pac&showCol%5B%5D=sho&showCol%5B%5D=pas&showCol%5B%5D=dri&showCol%5B%5D=def&showCol%5B%5D=phy')
html = BeautifulSoup(raw_html, 'html.parser')

# next page = https://sofifa.com/players?showCol%5B0%5D=pi&showCol%5B1%5D=ae&showCol%5B2%5D=hi&showCol%5B3%5D=wi&showCol%5B4%5D=pf&showCol%5B5%5D=oa&showCol%5B6%5D=pt&showCol%5B7%5D=bo&showCol%5B8%5D=bp&showCol%5B9%5D=gu&showCol%5B10%5D=jt&showCol%5B11%5D=le&showCol%5B12%5D=vl&showCol%5B13%5D=wg&showCol%5B14%5D=rc&showCol%5B15%5D=ta&showCol%5B16%5D=cr&showCol%5B17%5D=fi&showCol%5B18%5D=he&showCol%5B19%5D=sh&showCol%5B20%5D=vo&showCol%5B21%5D=ts&showCol%5B22%5D=dr&showCol%5B23%5D=cu&showCol%5B24%5D=fr&showCol%5B25%5D=lo&showCol%5B26%5D=bl&showCol%5B27%5D=to&showCol%5B28%5D=ac&showCol%5B29%5D=sp&showCol%5B30%5D=ag&showCol%5B31%5D=re&showCol%5B32%5D=ba&showCol%5B33%5D=tp&showCol%5B34%5D=so&showCol%5B35%5D=ju&showCol%5B36%5D=st&showCol%5B37%5D=sr&showCol%5B38%5D=ln&showCol%5B39%5D=te&showCol%5B40%5D=ar&showCol%5B41%5D=in&showCol%5B42%5D=po&showCol%5B43%5D=vi&showCol%5B44%5D=pe&showCol%5B45%5D=cm&showCol%5B46%5D=td&showCol%5B47%5D=ma&showCol%5B48%5D=sa&showCol%5B49%5D=sl&showCol%5B50%5D=tg&showCol%5B51%5D=gd&showCol%5B52%5D=gh&showCol%5B53%5D=gk&showCol%5B54%5D=gp&showCol%5B55%5D=gr&showCol%5B56%5D=tt&showCol%5B57%5D=bs&showCol%5B58%5D=wk&showCol%5B59%5D=sk&showCol%5B60%5D=aw&showCol%5B61%5D=dw&showCol%5B62%5D=ir&showCol%5B63%5D=pac&showCol%5B64%5D=sho&showCol%5B65%5D=pas&showCol%5B66%5D=dri&showCol%5B67%5D=def&showCol%5B68%5D=phy&offset=60

# specific player - https://sofifa.com/player/158023/lionel-messi/19/159502/

In [7]:
def get_rows(page_url):
    raw_html = simple_get(page_url)
    html = BeautifulSoup(raw_html, 'html.parser')
    
    rows = html.select('tr')
    return rows[2:]

def get_tds(row):
    return row.select('td')

In [8]:
def extract_name_url_positions_nation(player_info, data_dict):
    anchor = player_info.findAll('a', {'class' : 'nowrap'})[0]
    data_dict['Full Name'] = player_info.findAll('a', {'class' : 'nowrap'})[0]['title']
    data_dict['URL'] = sofifa_url + player_info.findAll('a', {'class' : 'nowrap'})[0]['href']
    
    positions = player_info.findAll('span', {'class' : 'pos'})
    positions = [pos.text for pos in positions]
    positions = list(np.unique(positions))
    data_dict['Positions'] = positions
    
    data_dict['Nation'] = player_info.findAll('a', {'rel' : 'nofollow'})[0]['title']
    
    return

In [9]:
def extract_contractstartend_teamname(team_contract_info, data_dict):
    data_dict['Team'] = team_contract_info.findAll('a')[0].text
    
    text = team_contract_info.findAll('div')[0].text.split()
    # figure out how to split if does not have ~
    if '~' in text:
        data_dict['Contract Start'] = text[-3]
        data_dict['Contract End'] = text[-1]
    else:
        data_dict['Contract Start'] = None
        data_dict['Contract End'] = ' '.join(text[-3:])

In [10]:
def extract_age(row, data_dict):
    data_dict['Age'] = row.findAll('td', {'class' : 'col-ae'})[0].text

In [11]:
def extract_overall_potential(row, data_dict):
    data_dict['Overall'] = row.findAll('td', {'class' : 'col-oa'})[0].find('span').text
    data_dict['Potential'] = row.findAll('td', {'class' : 'col-pt'})[0].find('span').text

In [12]:
def extract_basic(row, data_dict):
    data_dict['Id'] = row.findAll('td', {'class' : 'col-pi'})[0].text
    data_dict['Height'] = row.findAll('td', {'class' : 'col-hi'})[0].text
    data_dict['Weight'] = row.findAll('td', {'class' : 'col-wi'})[0].text
    data_dict['Foot'] = row.findAll('td', {'class' : 'col-pf'})[0].text
    data_dict['Best Overall'] = row.findAll('td', {'class' : 'col-bo'})[0].find('span').text
    data_dict['Best Position'] = row.findAll('td', {'class' : 'col-bp'})[0].find('span').text
    data_dict['Joined'] = row.findAll('td', {'class' : 'col-jt'})[0].text
    data_dict['Loan Date End'] = row.findAll('td', {'class' : 'col-le'})[0].text
    data_dict['Value'] = row.findAll('td', {'class' : 'col-vl'})[0].text
    data_dict['Wage'] = row.findAll('td', {'class' : 'col-wg'})[0].text
    data_dict['Release Clause'] = row.findAll('td', {'class' : 'col-rc'})[0].text

In [13]:
def extract_attacking(row, data_dict):
    data_dict['Total Attacking'] = row.findAll('td', {'class' : 'col-ta'})[0].find('span').text
    data_dict['Crossing'] = row.findAll('td', {'class' : 'col-cr'})[0].find('span').text
    data_dict['Finishing'] = row.findAll('td', {'class' : 'col-fi'})[0].find('span').text
    data_dict['Heading Accuracy'] = row.findAll('td', {'class' : 'col-he'})[0].find('span').text
    data_dict['Short Passing'] = row.findAll('td', {'class' : 'col-sh'})[0].find('span').text
    data_dict['Volleys'] = row.findAll('td', {'class' : 'col-vo'})[0].find('span').text

In [14]:
def extract_skill(row, data_dict):
    data_dict['Total Skill'] = row.findAll('td', {'class' : 'col-ts'})[0].find('span').text
    data_dict['Dribbling'] = row.findAll('td', {'class' : 'col-dr'})[0].find('span').text
    data_dict['Curve'] = row.findAll('td', {'class' : 'col-cu'})[0].find('span').text
    data_dict['FK Accuracy'] = row.findAll('td', {'class' : 'col-fr'})[0].find('span').text
    data_dict['Dribbling'] = row.findAll('td', {'class' : 'col-dr'})[0].find('span').text
    data_dict['Long Passing'] = row.findAll('td', {'class' : 'col-lo'})[0].find('span').text
    data_dict['Ball Control'] = row.findAll('td', {'class' : 'col-bl'})[0].find('span').text

In [15]:
def extract_movement(row, data_dict):
    data_dict['Total Movement'] = row.findAll('td', {'class' : 'col-to'})[0].find('span').text
    data_dict['Acceleration'] = row.findAll('td', {'class' : 'col-ac'})[0].find('span').text
    data_dict['Sprint Speed'] = row.findAll('td', {'class' : 'col-sp'})[0].find('span').text
    data_dict['Agility'] = row.findAll('td', {'class' : 'col-ag'})[0].find('span').text
    data_dict['Reactions'] = row.findAll('td', {'class' : 'col-re'})[0].find('span').text
    data_dict['Balance'] = row.findAll('td', {'class' : 'col-ba'})[0].find('span').text

In [16]:
def extract_power(row, data_dict):
    data_dict['Total Power'] = row.findAll('td', {'class' : 'col-tp'})[0].find('span').text
    data_dict['Shot Power'] = row.findAll('td', {'class' : 'col-so'})[0].find('span').text
    data_dict['Dribbling'] = row.findAll('td', {'class' : 'col-dr'})[0].find('span').text
    data_dict['Jumping'] = row.findAll('td', {'class' : 'col-ju'})[0].find('span').text
    data_dict['Stamina'] = row.findAll('td', {'class' : 'col-st'})[0].find('span').text
    data_dict['Strength'] = row.findAll('td', {'class' : 'col-sr'})[0].find('span').text
    data_dict['Dribbling'] = row.findAll('td', {'class' : 'col-dr'})[0].find('span').text
    data_dict['Long Shots'] = row.findAll('td', {'class' : 'col-ln'})[0].find('span').text

In [17]:
def extract_mentality(row, data_dict):
    data_dict['Total Mentality'] = row.findAll('td', {'class' : 'col-te'})[0].find('span').text
    data_dict['Aggression'] = row.findAll('td', {'class' : 'col-ar'})[0].find('span').text
    data_dict['Interceptions'] = row.findAll('td', {'class' : 'col-in'})[0].find('span').text
    data_dict['Positioning'] = row.findAll('td', {'class' : 'col-po'})[0].find('span').text
    data_dict['Vision'] = row.findAll('td', {'class' : 'col-vi'})[0].find('span').text
    data_dict['Penalties'] = row.findAll('td', {'class' : 'col-pe'})[0].find('span').text
    data_dict['Composure'] = row.findAll('td', {'class' : 'col-cm'})[0].find('span').text

In [18]:
def extract_defending(row, data_dict):
    data_dict['Total Defending'] = row.findAll('td', {'class' : 'col-td'})[0].find('span').text
    data_dict['Marking'] = row.findAll('td', {'class' : 'col-ma'})[0].find('span').text
    data_dict['Standing Tackle'] = row.findAll('td', {'class' : 'col-sa'})[0].find('span').text
    data_dict['Sliding Tackle'] = row.findAll('td', {'class' : 'col-sl'})[0].find('span').text

In [19]:
def extract_goalkeeping(row, data_dict):
    data_dict['Total Goalkeeping'] = row.findAll('td', {'class' : 'col-tg'})[0].find('span').text
    data_dict['GK Diving'] = row.findAll('td', {'class' : 'col-gd'})[0].find('span').text
    data_dict['GK Handling'] = row.findAll('td', {'class' : 'col-gh'})[0].find('span').text
    data_dict['GK Kicking'] = row.findAll('td', {'class' : 'col-gk'})[0].find('span').text
    data_dict['GK Positioning'] = row.findAll('td', {'class' : 'col-gp'})[0].find('span').text
    data_dict['GK Reflexes'] = row.findAll('td', {'class' : 'col-gr'})[0].find('span').text

In [20]:
def extract_other(row, data_dict):
    data_dict['Total Stats'] = row.findAll('td', {'class' : 'col-tt'})[0].find('span').text
    data_dict['Base Stats'] = row.findAll('td', {'class' : 'col-bs'})[0].find('span').text
    data_dict['Weak Foot'] = row.findAll('td', {'class' : 'col-wk'})[0].text
    data_dict['Skill Moves'] = row.findAll('td', {'class' : 'col-sk'})[0].text
    data_dict['Attacking Work Rate'] = row.findAll('td', {'class' : 'col-aw'})[0].text
    data_dict['Defensive Work Rate'] = row.findAll('td', {'class' : 'col-dw'})[0].text
    data_dict['International Reputation'] = row.findAll('td', {'class' : 'col-ir'})[0].text

In [21]:
def extract_ultimateteam(row, data_dict):
    data_dict['PAC'] = row.findAll('td', {'class' : 'col-pac'})[0].find('span').text
    data_dict['SHO'] = row.findAll('td', {'class' : 'col-sho'})[0].find('span').text
    data_dict['PAS'] = row.findAll('td', {'class' : 'col-pas'})[0].find('span').text
    data_dict['DRI'] = row.findAll('td', {'class' : 'col-dri'})[0].find('span').text
    data_dict['DEF'] = row.findAll('td', {'class' : 'col-def'})[0].find('span').text
    data_dict['PHY'] = row.findAll('td', {'class' : 'col-phy'})[0].find('span').text

In [22]:
players.columns

Index(['Full Name', 'Name', 'Age', 'Overall', 'Potential', 'Team', 'Nation',
       'Contract Start', 'Contract End', 'Id', 'Height', 'Weight', 'Foot',
       'Best Overall', 'Best Position', 'Joined', 'Loan Date End', 'Value',
       'Wage', 'Release Clause', 'Total Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Total Skill',
       'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control',
       'Total Movement', 'Acceleration', 'Sprint Speed', 'Agility',
       'Reactions', 'Balance', 'Total Power', 'Shot Power', 'Jumping',
       'Stamina', 'Strength', 'Long Shots', 'Total Mentality', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Total Defending', 'Marking', 'Standing Tackle', 'Sliding Tackle',
       'Total Goalkeeping', 'GK Diving', 'GK Handling', 'GK Kicking',
       'GK Positioning', 'GK Reflexes', 'Total Stats', 'Base Stats',
       'Weak Foot', 'Skill Moves', 'Attacking

In [23]:
base_url = 'https://sofifa.com/players?&showCol%5B%5D=pi&showCol%5B%5D=ae&showCol%5B%5D=hi&showCol%5B%5D=wi&showCol%5B%5D=pf&showCol%5B%5D=oa&showCol%5B%5D=pt&showCol%5B%5D=bo&showCol%5B%5D=bp&showCol%5B%5D=gu&showCol%5B%5D=jt&showCol%5B%5D=le&showCol%5B%5D=vl&showCol%5B%5D=wg&showCol%5B%5D=rc&showCol%5B%5D=ta&showCol%5B%5D=cr&showCol%5B%5D=fi&showCol%5B%5D=he&showCol%5B%5D=sh&showCol%5B%5D=vo&showCol%5B%5D=ts&showCol%5B%5D=dr&showCol%5B%5D=cu&showCol%5B%5D=fr&showCol%5B%5D=lo&showCol%5B%5D=bl&showCol%5B%5D=to&showCol%5B%5D=ac&showCol%5B%5D=sp&showCol%5B%5D=ag&showCol%5B%5D=re&showCol%5B%5D=ba&showCol%5B%5D=tp&showCol%5B%5D=so&showCol%5B%5D=ju&showCol%5B%5D=st&showCol%5B%5D=sr&showCol%5B%5D=ln&showCol%5B%5D=te&showCol%5B%5D=ar&showCol%5B%5D=in&showCol%5B%5D=po&showCol%5B%5D=vi&showCol%5B%5D=pe&showCol%5B%5D=cm&showCol%5B%5D=td&showCol%5B%5D=ma&showCol%5B%5D=sa&showCol%5B%5D=sl&showCol%5B%5D=tg&showCol%5B%5D=gd&showCol%5B%5D=gh&showCol%5B%5D=gk&showCol%5B%5D=gp&showCol%5B%5D=gr&showCol%5B%5D=tt&showCol%5B%5D=bs&showCol%5B%5D=wk&showCol%5B%5D=sk&showCol%5B%5D=aw&showCol%5B%5D=dw&showCol%5B%5D=ir&showCol%5B%5D=pac&showCol%5B%5D=sho&showCol%5B%5D=pas&showCol%5B%5D=dri&showCol%5B%5D=def&showCol%5B%5D=phy'
sofifa_url = 'https://sofifa.com'
offset = 0

count = 1
b = True
while b:
    count = 1
    if offset > 22000:
        break
    try:
        url = base_url
        if offset != 0:
            url += '&offset=' + str(offset)
        rows = get_rows(url)
        for row in rows:
            tds = get_tds(row)
            data_dict = dict()
    
            # first td - avatar (ignore)
            # col-name
            td_names = row.findAll('td', {'class' : 'col-name'})
            if len(td_names[0].findAll('span', {'class' : 'bp3-intent-success'})) > 0:
                print ('{} of {}'.format(count+offset, offset+60))
                count += 1
                continue
            # first one - name
            player_info = td_names[0]
            extract_name_url_positions_nation(player_info, data_dict)

            # second one = team and contract
            team_contract_info = td_names[1]
            extract_contractstartend_teamname(team_contract_info, data_dict)
            extract_age(row, data_dict)
            extract_overall_potential(row, data_dict)
            extract_basic(row, data_dict)
            extract_attacking(row, data_dict)
            extract_skill(row, data_dict)
            extract_movement(row, data_dict)
            extract_power(row, data_dict)
            extract_mentality(row, data_dict)
            extract_defending(row, data_dict)
            extract_goalkeeping(row, data_dict)
            extract_other(row, data_dict)
            extract_ultimateteam(row, data_dict)
            
            
#             name = data_dict['Name']
#             if (players.loc[(players['Name'] == name) & (players['Team'] == data_dict['Team']) & (players['Age'] == data_dict['Age'])].any().any()):
#                 print ('here')
#                 b = False
#                 break

            print ('{} of {}'.format(count+offset, offset+60))
            count += 1
            players = players.append(data_dict, ignore_index=True, sort=True)
        offset += 60
    except Exception as e:
        print (e)
        break

1 of 60
2 of 60
3 of 60
4 of 60
5 of 60
6 of 60
7 of 60
8 of 60
9 of 60
10 of 60
11 of 60
12 of 60
13 of 60
14 of 60
15 of 60
16 of 60
17 of 60
18 of 60
19 of 60
20 of 60
21 of 60
22 of 60
23 of 60
24 of 60
25 of 60
26 of 60
27 of 60
28 of 60
29 of 60
30 of 60
31 of 60
32 of 60
33 of 60
34 of 60
35 of 60
36 of 60
37 of 60
38 of 60
39 of 60
40 of 60
41 of 60
42 of 60
43 of 60
44 of 60
45 of 60
46 of 60
47 of 60
48 of 60
49 of 60
50 of 60
51 of 60
52 of 60
53 of 60
54 of 60
55 of 60
56 of 60
57 of 60
58 of 60
59 of 60
60 of 60
61 of 120
62 of 120
63 of 120
64 of 120
65 of 120
66 of 120
67 of 120
68 of 120
69 of 120
70 of 120
71 of 120
72 of 120
73 of 120
74 of 120
75 of 120
76 of 120
77 of 120
78 of 120
79 of 120
80 of 120
81 of 120
82 of 120
83 of 120
84 of 120
85 of 120
86 of 120
87 of 120
88 of 120
89 of 120
90 of 120
91 of 120
92 of 120
93 of 120
94 of 120
95 of 120
96 of 120
97 of 120
98 of 120
99 of 120
100 of 120
101 of 120
102 of 120
103 of 120
104 of 120
105 of 120
106 of 120
10

764 of 780
765 of 780
766 of 780
767 of 780
768 of 780
769 of 780
770 of 780
771 of 780
772 of 780
773 of 780
774 of 780
775 of 780
776 of 780
777 of 780
778 of 780
779 of 780
780 of 780
781 of 840
782 of 840
783 of 840
784 of 840
785 of 840
786 of 840
787 of 840
788 of 840
789 of 840
790 of 840
791 of 840
792 of 840
793 of 840
794 of 840
795 of 840
796 of 840
797 of 840
798 of 840
799 of 840
800 of 840
801 of 840
802 of 840
803 of 840
804 of 840
805 of 840
806 of 840
807 of 840
808 of 840
809 of 840
810 of 840
811 of 840
812 of 840
813 of 840
814 of 840
815 of 840
816 of 840
817 of 840
818 of 840
819 of 840
820 of 840
821 of 840
822 of 840
823 of 840
824 of 840
825 of 840
826 of 840
827 of 840
828 of 840
829 of 840
830 of 840
831 of 840
832 of 840
833 of 840
834 of 840
835 of 840
836 of 840
837 of 840
838 of 840
839 of 840
840 of 840
841 of 900
842 of 900
843 of 900
844 of 900
845 of 900
846 of 900
847 of 900
848 of 900
849 of 900
850 of 900
851 of 900
852 of 900
853 of 900
854 of 900

1429 of 1440
1430 of 1440
1431 of 1440
1432 of 1440
1433 of 1440
1434 of 1440
1435 of 1440
1436 of 1440
1437 of 1440
1438 of 1440
1439 of 1440
1440 of 1440
1441 of 1500
1442 of 1500
1443 of 1500
1444 of 1500
1445 of 1500
1446 of 1500
1447 of 1500
1448 of 1500
1449 of 1500
1450 of 1500
1451 of 1500
1452 of 1500
1453 of 1500
1454 of 1500
1455 of 1500
1456 of 1500
1457 of 1500
1458 of 1500
1459 of 1500
1460 of 1500
1461 of 1500
1462 of 1500
1463 of 1500
1464 of 1500
1465 of 1500
1466 of 1500
1467 of 1500
1468 of 1500
1469 of 1500
1470 of 1500
1471 of 1500
1472 of 1500
1473 of 1500
1474 of 1500
1475 of 1500
1476 of 1500
1477 of 1500
1478 of 1500
1479 of 1500
1480 of 1500
1481 of 1500
1482 of 1500
1483 of 1500
1484 of 1500
1485 of 1500
1486 of 1500
1487 of 1500
1488 of 1500
1489 of 1500
1490 of 1500
1491 of 1500
1492 of 1500
1493 of 1500
1494 of 1500
1495 of 1500
1496 of 1500
1497 of 1500
1498 of 1500
1499 of 1500
1500 of 1500
1501 of 1560
1502 of 1560
1503 of 1560
1504 of 1560
1505 of 1560

2062 of 2100
2063 of 2100
2064 of 2100
2065 of 2100
2066 of 2100
2067 of 2100
2068 of 2100
2069 of 2100
2070 of 2100
2071 of 2100
2072 of 2100
2073 of 2100
2074 of 2100
2075 of 2100
2076 of 2100
2077 of 2100
2078 of 2100
2079 of 2100
2080 of 2100
2081 of 2100
2082 of 2100
2083 of 2100
2084 of 2100
2085 of 2100
2086 of 2100
2087 of 2100
2088 of 2100
2089 of 2100
2090 of 2100
2091 of 2100
2092 of 2100
2093 of 2100
2094 of 2100
2095 of 2100
2096 of 2100
2097 of 2100
2098 of 2100
2099 of 2100
2100 of 2100
2101 of 2160
2102 of 2160
2103 of 2160
2104 of 2160
2105 of 2160
2106 of 2160
2107 of 2160
2108 of 2160
2109 of 2160
2110 of 2160
2111 of 2160
2112 of 2160
2113 of 2160
2114 of 2160
2115 of 2160
2116 of 2160
2117 of 2160
2118 of 2160
2119 of 2160
2120 of 2160
2121 of 2160
2122 of 2160
2123 of 2160
2124 of 2160
2125 of 2160
2126 of 2160
2127 of 2160
2128 of 2160
2129 of 2160
2130 of 2160
2131 of 2160
2132 of 2160
2133 of 2160
2134 of 2160
2135 of 2160
2136 of 2160
2137 of 2160
2138 of 2160

2694 of 2700
2695 of 2700
2696 of 2700
2697 of 2700
2698 of 2700
2699 of 2700
2700 of 2700
2701 of 2760
2702 of 2760
2703 of 2760
2704 of 2760
2705 of 2760
2706 of 2760
2707 of 2760
2708 of 2760
2709 of 2760
2710 of 2760
2711 of 2760
2712 of 2760
2713 of 2760
2714 of 2760
2715 of 2760
2716 of 2760
2717 of 2760
2718 of 2760
2719 of 2760
2720 of 2760
2721 of 2760
2722 of 2760
2723 of 2760
2724 of 2760
2725 of 2760
2726 of 2760
2727 of 2760
2728 of 2760
2729 of 2760
2730 of 2760
2731 of 2760
2732 of 2760
2733 of 2760
2734 of 2760
2735 of 2760
2736 of 2760
2737 of 2760
2738 of 2760
2739 of 2760
2740 of 2760
2741 of 2760
2742 of 2760
2743 of 2760
2744 of 2760
2745 of 2760
2746 of 2760
2747 of 2760
2748 of 2760
2749 of 2760
2750 of 2760
2751 of 2760
2752 of 2760
2753 of 2760
2754 of 2760
2755 of 2760
2756 of 2760
2757 of 2760
2758 of 2760
2759 of 2760
2760 of 2760
2761 of 2820
2762 of 2820
2763 of 2820
2764 of 2820
2765 of 2820
2766 of 2820
2767 of 2820
2768 of 2820
2769 of 2820
2770 of 2820

...

19560 of 19560
19561 of 19620
19562 of 19620
19563 of 19620
19564 of 19620
19565 of 19620
19566 of 19620
19567 of 19620
19568 of 19620
19569 of 19620
19570 of 19620
19571 of 19620
19572 of 19620
19573 of 19620
19574 of 19620
19575 of 19620
19576 of 19620
19577 of 19620
19578 of 19620
19579 of 19620
19580 of 19620
19581 of 19620
19582 of 19620
19583 of 19620
19584 of 19620
19585 of 19620
19586 of 19620
19587 of 19620
19588 of 19620
19589 of 19620
19590 of 19620
19591 of 19620
19592 of 19620
19593 of 19620
19594 of 19620
19595 of 19620
19596 of 19620
19597 of 19620
19598 of 19620
19599 of 19620
19600 of 19620
19601 of 19620
19602 of 19620
19603 of 19620
19604 of 19620
19605 of 19620
19606 of 19620
19607 of 19620
19608 of 19620
19609 of 19620
19610 of 19620
19611 of 19620
19612 of 19620
19613 of 19620
19614 of 19620
19615 of 19620
19616 of 19620
19617 of 19620
19618 of 19620
19619 of 19620
19620 of 19620
19621 of 19680
19622 of 19680
19623 of 19680
19624 of 19680
19625 of 19680
19626 of 1

20108 of 20160
20109 of 20160
20110 of 20160
20111 of 20160
20112 of 20160
20113 of 20160
20114 of 20160
20115 of 20160
20116 of 20160
20117 of 20160
20118 of 20160
20119 of 20160
20120 of 20160
20121 of 20160
20122 of 20160
20123 of 20160
20124 of 20160
20125 of 20160
20126 of 20160
20127 of 20160
20128 of 20160
20129 of 20160
20130 of 20160
20131 of 20160
20132 of 20160
20133 of 20160
20134 of 20160
20135 of 20160
20136 of 20160
20137 of 20160
20138 of 20160
20139 of 20160
20140 of 20160
20141 of 20160
20142 of 20160
20143 of 20160
20144 of 20160
20145 of 20160
20146 of 20160
20147 of 20160
20148 of 20160
20149 of 20160
20150 of 20160
20151 of 20160
20152 of 20160
20153 of 20160
20154 of 20160
20155 of 20160
20156 of 20160
20157 of 20160
20158 of 20160
20159 of 20160
20160 of 20160
20161 of 20220
20162 of 20220
20163 of 20220
20164 of 20220
20165 of 20220
20166 of 20220
20167 of 20220
20168 of 20220
20169 of 20220
20170 of 20220
20171 of 20220
20172 of 20220
20173 of 20220
20174 of 2

20655 of 20700
20656 of 20700
20657 of 20700
20658 of 20700
20659 of 20700
20660 of 20700
20661 of 20700
20662 of 20700
20663 of 20700
20664 of 20700
20665 of 20700
20666 of 20700
20667 of 20700
20668 of 20700
20669 of 20700
20670 of 20700
20671 of 20700
20672 of 20700
20673 of 20700
20674 of 20700
20675 of 20700
20676 of 20700
20677 of 20700
20678 of 20700
20679 of 20700
20680 of 20700
20681 of 20700
20682 of 20700
20683 of 20700
20684 of 20700
20685 of 20700
20686 of 20700
20687 of 20700
20688 of 20700
20689 of 20700
20690 of 20700
20691 of 20700
20692 of 20700
20693 of 20700
20694 of 20700
20695 of 20700
20696 of 20700
20697 of 20700
20698 of 20700
20699 of 20700
20700 of 20700
20701 of 20760
20702 of 20760
20703 of 20760
20704 of 20760
20705 of 20760
20706 of 20760
20707 of 20760
20708 of 20760
20709 of 20760
20710 of 20760
20711 of 20760
20712 of 20760
20713 of 20760
20714 of 20760
20715 of 20760
20716 of 20760
20717 of 20760
20718 of 20760
20719 of 20760
20720 of 20760
20721 of 2

21202 of 21240
21203 of 21240
21204 of 21240
21205 of 21240
21206 of 21240
21207 of 21240
21208 of 21240
21209 of 21240
21210 of 21240
21211 of 21240
21212 of 21240
21213 of 21240
21214 of 21240
21215 of 21240
21216 of 21240
21217 of 21240
21218 of 21240
21219 of 21240
21220 of 21240
21221 of 21240
21222 of 21240
21223 of 21240
21224 of 21240
21225 of 21240
21226 of 21240
21227 of 21240
21228 of 21240
21229 of 21240
21230 of 21240
21231 of 21240
21232 of 21240
21233 of 21240
21234 of 21240
21235 of 21240
21236 of 21240
21237 of 21240
21238 of 21240
21239 of 21240
21240 of 21240
21241 of 21300
21242 of 21300
21243 of 21300
21244 of 21300
21245 of 21300
21246 of 21300
21247 of 21300
21248 of 21300
21249 of 21300
21250 of 21300
21251 of 21300
21252 of 21300
21253 of 21300
21254 of 21300
21255 of 21300
21256 of 21300
21257 of 21300
21258 of 21300
21259 of 21300
21260 of 21300
21261 of 21300
21262 of 21300
21263 of 21300
21264 of 21300
21265 of 21300
21266 of 21300
21267 of 21300
21268 of 2

21749 of 21780
21750 of 21780
21751 of 21780
21752 of 21780
21753 of 21780
21754 of 21780
21755 of 21780
21756 of 21780
21757 of 21780
21758 of 21780
21759 of 21780
21760 of 21780
21761 of 21780
21762 of 21780
21763 of 21780
21764 of 21780
21765 of 21780
21766 of 21780
21767 of 21780
21768 of 21780
21769 of 21780
21770 of 21780
21771 of 21780
21772 of 21780
21773 of 21780
21774 of 21780
21775 of 21780
21776 of 21780
21777 of 21780
21778 of 21780
21779 of 21780
21780 of 21780
21781 of 21840
21782 of 21840
21783 of 21840
21784 of 21840
21785 of 21840
21786 of 21840
21787 of 21840
21788 of 21840
21789 of 21840
21790 of 21840
21791 of 21840
21792 of 21840
21793 of 21840
21794 of 21840
21795 of 21840
21796 of 21840
21797 of 21840
21798 of 21840
21799 of 21840
21800 of 21840
21801 of 21840
21802 of 21840
21803 of 21840
21804 of 21840
21805 of 21840
21806 of 21840
21807 of 21840
21808 of 21840
21809 of 21840
21810 of 21840
21811 of 21840
21812 of 21840
21813 of 21840
21814 of 21840
21815 of 2

In [24]:
players.head()

,Acceleration,Age,Aggression,Agility,Attacking Work Rate,Balance,Ball Control,Base Stats,Best Overall,Best Position,Composure,Contract End,Contract Start,Crossing,Curve,DEF,DRI,Defensive Work Rate,Dribbling,FK Accuracy,Finishing,Foot,Full Name,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Heading Accuracy,Height,Id,Interceptions,International Reputation,Joined,Jumping,Loan Date End,Long Passing,Long Shots,Marking,Name,Nation,Overall,PAC,PAS,PHY,Penalties,Positioning,Positions,Potential,Reactions,Release Clause,SHO,Short Passing,Shot Power,Skill Moves,Sliding Tackle,Sprint Speed,Stamina,Standing Tackle,Strength,Team,Total Attacking,Total Defending,Total Goalkeeping,Total Mentality,Total Movement,Total Power,Total Skill,Total Stats,URL,Value,Vision,Volleys,Wage,Weak Foot,Weight
0,73,19,81,61,Medium,62,64,391,78,CB,68,2021,2017,47,32,75,61,Medium,59,28,39,Right,Dayot Upamecano,6,7,8,15,9,71,"6'1""",229558,75,1,"Jan 13, 2017",86,N/A,66,24,68,NaN,France,77,80,59,80,39,35,[CB],88,67,€24.7M,36,72,44,2,75,85,61,84,88,RB Leipzig,250,227,45,286,348,303,249,1708,https://sofifa.com/player/229558/dayot-upameca...,€13M,56,21,€18K,3,183lbs
1,68,21,80,50,High,45,78,413,78,ST,78,2023,2016,68,57,35,71,High,74,49,73,Right,Wesley Moraes Ferreira da Silva,12,13,14,8,6,73,"6'3""",232381,37,1,"Jan 29, 2016",68,N/A,70,73,35,NaN,Brazil,78,76,69,87,60,77,[ST],85,74,€24M,75,75,84,3,15,83,81,27,95,Club Brugge KV,362,77,53,322,320,401,328,1863,https://sofifa.com/player/232381/wesley-moraes...,€15M,68,73,€21K,3,207lbs
2,81,19,52,82,High,84,77,378,74,CAM,65,2022,2015,68,72,38,78,Medium,78,69,64,Left,Abdülkadir Ömür,11,12,13,11,10,50,"5'6""",231777,35,1,"Jul 20, 2015",53,N/A,62,64,35,NaN,Turkey,73,81,70,47,55,67,"[LM, RM]",84,67,€14.3M,64,72,68,3,35,81,67,40,34,Trabzonspor,311,110,57,282,395,286,358,1799,https://sofifa.com/player/231777/abdulkadir-om...,€6.5M,73,57,€12K,3,148lbs
3,89,18,41,90,Medium,89,86,403,82,RW,74,2022,2017,73,77,32,88,Low,89,48,73,Right,Jadon Sancho,7,11,10,11,13,38,"5'11""",233049,28,2,"Aug 31, 2017",51,N/A,57,52,30,NaN,England,80,88,72,56,57,73,"[LM, RM]",91,79,€44.7M,67,77,65,5,36,87,73,33,54,Borussia Dortmund,337,99,52,277,434,295,357,1851,https://sofifa.com/player/233049/jadon-sancho/...,€23.5M,78,76,€28K,4,168lbs
4,75,18,45,75,Medium,69,76,368,74,ST,72,2020,2016,42,66,29,75,Medium,75,43,75,Right,Moise Kean,13,8,10,7,9,65,"6'0""",236610,21,1,"Aug 1, 2016",68,N/A,38,58,27,NaN,Italy,73,77,50,67,65,74,[ST],87,72,€19.3M,70,59,71,3,16,79,69,28,75,Juventus,309,71,47,256,370,341,298,1692,https://sofifa.com/player/236610/moise-kean/19...,€8.5M,51,68,€36K,3,174lbs


In [25]:
players_nopos = players[['Acceleration', 'Age', 'Aggression', 'Agility', 'Attacking Work Rate',
       'Balance', 'Ball Control', 'Base Stats', 'Best Overall',
       'Best Position', 'Composure', 'Contract End', 'Contract Start',
       'Crossing', 'Curve', 'DEF', 'DRI', 'Defensive Work Rate', 'Dribbling',
       'FK Accuracy', 'Finishing', 'Foot', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Heading Accuracy',
       'Height', 'Id', 'Interceptions', 'International Reputation', 'Joined',
       'Jumping', 'Loan Date End', 'Long Passing', 'Long Shots', 'Marking',
       'Name', 'Nation', 'Overall', 'PAC', 'PAS', 'PHY', 'Penalties',
       'Positioning', 'Potential', 'Reactions', 'Release Clause',
       'SHO', 'Short Passing', 'Shot Power', 'Skill Moves', 'Sliding Tackle',
       'Sprint Speed', 'Stamina', 'Standing Tackle', 'Strength', 'Team',
       'Total Attacking', 'Total Defending', 'Total Goalkeeping',
       'Total Mentality', 'Total Movement', 'Total Power', 'Total Skill',
       'Total Stats', 'URL', 'Value', 'Vision', 'Volleys', 'Wage', 'Weak Foot',
       'Weight']]

In [26]:
players_nodups = players.drop_duplicates(subset=['Acceleration', 'Age', 'Aggression', 'Agility', 'Attacking Work Rate',
       'Balance', 'Ball Control', 'Base Stats', 'Best Overall',
       'Best Position', 'Composure', 'Contract End', 'Contract Start',
       'Crossing', 'Curve', 'DEF', 'DRI', 'Defensive Work Rate', 'Dribbling',
       'FK Accuracy', 'Finishing', 'Foot', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Heading Accuracy',
       'Height', 'Id', 'Interceptions', 'International Reputation', 'Joined',
       'Jumping', 'Loan Date End', 'Long Passing', 'Long Shots', 'Marking',
       'Name', 'Nation', 'Overall', 'PAC', 'PAS', 'PHY', 'Penalties',
       'Positioning', 'Potential', 'Reactions', 'Release Clause',
       'SHO', 'Short Passing', 'Shot Power', 'Skill Moves', 'Sliding Tackle',
       'Sprint Speed', 'Stamina', 'Standing Tackle', 'Strength', 'Team',
       'Total Attacking', 'Total Defending', 'Total Goalkeeping',
       'Total Mentality', 'Total Movement', 'Total Power', 'Total Skill',
       'Total Stats', 'URL', 'Value', 'Vision', 'Volleys', 'Wage', 'Weak Foot',
       'Weight'])

In [27]:
players_nodups.shape

(12433, 76)

In [28]:
players_nodups.to_csv('PlayerDataFull.csv',index=False)

In [29]:
players.tail()

,Acceleration,Age,Aggression,Agility,Attacking Work Rate,Balance,Ball Control,Base Stats,Best Overall,Best Position,Composure,Contract End,Contract Start,Crossing,Curve,DEF,DRI,Defensive Work Rate,Dribbling,FK Accuracy,Finishing,Foot,Full Name,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Heading Accuracy,Height,Id,Interceptions,International Reputation,Joined,Jumping,Loan Date End,Long Passing,Long Shots,Marking,Name,Nation,Overall,PAC,PAS,PHY,Penalties,Positioning,Positions,Potential,Reactions,Release Clause,SHO,Short Passing,Shot Power,Skill Moves,Sliding Tackle,Sprint Speed,Stamina,Standing Tackle,Strength,Team,Total Attacking,Total Defending,Total Goalkeeping,Total Mentality,Total Movement,Total Power,Total Skill,Total Stats,URL,Value,Vision,Volleys,Wage,Weak Foot,Weight
15296,92,17,68,84,High,76,73,420,73,LW,71,2023,2019,62,64,55,75,High,76,44,64,Left,Alphonso Davies,11,11,6,5,14,51,"5'11""",234396,52,1,"Jan 1, 2019",72,N/A,51,63,55,NaN,Canada,72,93,61,71,60,65,"[LM, RM]",87,65,€13.7M,65,64,71,4,51,93,76,61,69,FC Bayern München,296,167,47,310,410,351,308,1889,https://sofifa.com/player/234396/alphonso-davi...,€6M,65,55,€20K,4,159lbs
15297,91,31,48,93,Medium,95,96,462,95,CAM,96,2021,2004,86,93,32,96,Low,97,94,95,Left,Lionel Messi,6,11,15,14,8,70,"5'7""",158023,22,5,"Jul 1, 2004",68,N/A,89,94,33,NaN,Argentina,94,88,91,64,75,94,"[CF, RW, ST]",94,95,€226.5M,91,92,85,4,26,86,72,28,66,FC Barcelona,429,87,54,333,460,385,469,2217,https://sofifa.com/player/158023/lionel-messi/...,€110.5M,94,86,€565K,4,159lbs
15298,69,22,68,73,Medium,66,83,449,82,CM,77,2023,2018,72,75,74,81,Medium,83,59,79,Left,Fabián Ruiz Peña,16,11,7,13,13,67,"6'2""",226271,77,1,"Jul 5, 2018",46,N/A,80,83,70,NaN,Spain,82,68,78,70,67,80,"[CAM, CDM, CM]",90,79,€60.8M,78,80,75,4,75,67,75,78,70,Napoli,365,223,60,376,354,349,380,2107,https://sofifa.com/player/226271/fabian-ruiz-p...,€32M,84,67,€70K,4,154lbs
15299,77,23,75,68,High,70,79,446,82,LWB,79,2022,2017,87,80,78,78,Low,80,34,33,Left,Benjamin Mendy,7,7,9,15,9,65,"6'1""",204884,79,2,"Jul 24, 2017",71,N/A,69,65,78,NaN,France,81,82,77,79,50,77,[LB],86,80,€38.5M,52,80,79,3,81,86,79,79,81,Manchester City,296,238,47,357,381,375,342,2036,https://sofifa.com/player/204884/benjamin-mend...,€20M,76,31,€105K,3,187lbs
15300,81,21,78,83,Medium,78,74,412,79,RWB,74,2022,2015,79,76,76,75,Medium,73,48,30,Right,Davide Calabria,8,14,15,12,7,64,"5'9""",228881,77,1,"Jul 1, 2015",69,N/A,68,48,78,NaN,Italy,78,79,68,71,48,42,[RB],88,74,€27.6M,43,75,62,3,77,77,81,78,64,Milan,306,233,56,295,393,324,339,1946,https://sofifa.com/player/228881/davide-calabr...,€14.5M,50,58,€52K,3,154lbs
